In [1]:
## EDIT:
base_dir = '/glade/work/kjmayer/research/catalyst/TransferLearning/runmean_analysis/artificial_bias/perfectmodel_TLtest/E3SM_analysis/'
import xarray as xr
import numpy as np
import datetime as dt
import pandas as pd
import time

import sys
sys.path.append(base_dir+'functions/')
from utils import detrend_members, is_month

In [2]:
# get nino3.4 data (not detrended):
# old path that had been overwritten:
# path = '/glade/campaign/cgd/amp/jcaron/ongoing_projects/cvdp-le/output/final_cesm_e3sm_future_nonprime/1920-2100/'
path = '/glade/campaign/cgd/amp/jcaron/ongoing_projects/cvdp-le/output/for_kirsten/1920-2100/'
ens = ['0101','0111','0121','0131','0141','0151','0161','0171','0181','0191',
       '0201','0211','0221','0231','0241','0251','0261','0271','0281','0291','0301']

for e in range(len(ens)):
    finame = 'E3SM2_'+ens[e]+'.cvdp_data.1920-2100.nc'
    if e == 0:
        nino34 = xr.open_dataset(path+finame,decode_times=False)['nino34']
    elif e > 0:
        temp_nino34 = xr.open_dataset(path+finame,decode_times=False)['nino34']
        nino34 = xr.concat([nino34,temp_nino34],dim='ens')

newtime = pd.date_range("1920-01-01", freq="MS", periods=len(nino34.time))
nino34['time'] = newtime                           
nino34 = nino34.where((nino34.time.dt.year >= 1950) & (nino34.time.dt.year <= 2015), drop = True)

# nino34[0].plot()

nino34_ensmean = nino34.mean('ens')
nino34_detrended = detrend_members(data = nino34, 
                                   ensmean_data = nino34_ensmean, 
                                   npoly=3)
nino34_detrended_upsample = nino34_detrended.resample(time='1D').asfreq()
nino34_detrended_upsample = nino34_detrended_upsample.where(~((nino34_detrended_upsample.time.dt.month == 2) & (nino34_detrended_upsample.time.dt.day == 29)),drop=True)
nino34_detrended_upsample = nino34_detrended_upsample.ffill(dim='time')
nino34_detrended_upsample = nino34_detrended_upsample[:,6:-335] # 6: account for days 1-6 missing in X because of rolling mean; 
                                                                #-355: had to go through 2015 (not 2014) to get all days in Dec 2014 (not just 12-01-2014)

In [17]:
lead = 14
ensofrac = []
ninafrac = []
ninofrac = []
for memind in range(len(ens)):
    nino34_test = nino34_detrended_upsample[memind][:-6][:-lead]
    nino34_NDJF_test = is_month(nino34_test, [11,12,1,2])
    # nino34_NDJF_test = nino34_NDJF_test.isel(time = inew,drop=True) 
    
    ienso = np.where(np.abs(nino34_NDJF_test) > 1)[0]
    inino = np.where(nino34_NDJF_test > 1)[0]
    inina = np.where(nino34_NDJF_test < -1)[0]

    ensofrac.append(np.shape(ienso)[0]/np.shape(nino34_NDJF_test)[0])
    ninofrac.append(np.shape(inino)[0]/np.shape(nino34_NDJF_test)[0]) #fraction of days with el nino
    ninafrac.append(np.shape(inina)[0]/np.shape(nino34_NDJF_test)[0]) #fraction of days with la nina

print(np.mean(ensofrac))
print(np.mean(ninofrac))
print(np.mean(ninafrac))

0.4313217440307741
0.20964264275301064
0.22167910127776352
